In [2]:
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
from koopmanlib.dictionary import PsiNN
from koopmanlib.solver import KoopmanDLSolver

In [3]:
dim = 1 
hidden_dim = 80
input_dim = 1
n_feature = 19
num_epochs = 100
batch_size = 128
n_train = 4140
n_test = 4140
learning_rate = 1e-3

In [4]:
X_train = pd.read_csv('EMPS_X_train.csv', header=None).values
X_test = pd.read_csv('EMPS_X_test.csv', header=None).values
U_train = pd.read_csv('EMPS_U_train.csv', header=None).values
U_test = pd.read_csv('EMPS_U_test.csv', header=None).values

length = X_train.shape[1] // n_train
HX_train = []
HU_train = []
for i in range(n_train):
    HX_train.append(X_train[:, i*length:(i+1)*length])
    HU_train.append(U_train[:, i*length:(i+1)*length])
HX_train = np.stack([HX_train[idx].T for idx in range(n_train)], axis=0)
HU_train = np.stack([HU_train[idx].T for idx in range(n_train)], axis=0)
HX_test = []
HU_test = []
for i in range(n_test):
    HX_test.append(X_test[:, i*length:(i+1)*length])
    HU_test.append(U_test[:, i*length:(i+1)*length])
HX_test = np.stack([HX_test[idx].T for idx in range(n_test)], axis=0)
HU_test = np.stack([HU_test[idx].T for idx in range(n_test)], axis=0)

H_train = np.concatenate([HX_train, HU_train], axis=-1)
H_test = np.concatenate([HX_test, HU_test], axis=-1)

In [ ]:
error_temp = []

data_train = [H_train[:, 0:5,:].reshape(-1, dim+input_dim), H_train[:, 1:6,:].reshape(-1, dim+input_dim)]
data_valid = [H_train[:, 0:5,:].reshape(-1, dim+input_dim), H_train[:, 1:6,:].reshape(-1, dim+input_dim)]
basis_function = PsiNN(layer_sizes=[hidden_dim, hidden_dim, hidden_dim], n_psi_train=dim+n_feature)
solver = KoopmanDLSolver(dic=basis_function,
                        target_dim=dim+input_dim,
                        reg=0.0)
solver.build(data_train=data_train,
                 data_valid=data_valid,
                 epochs=num_epochs,
                 batch_size=batch_size,
                 lr=learning_rate,
                 log_interval=1,
                 lr_decay_factor=0.92)
for j in range(n_test):
    x_traj = H_test[j, 0:6,:]
    x0_test = x_traj[0]
    x0_test = x0_test.reshape(-1, x0_test.shape[-1])
    x_est_traj_DL = solver.predict(x0_test, 5)
    error_temp.append((x_est_traj_DL[:, :dim]-x_traj[1:, :dim]) ** 2)
error = np.mean(np.array(error_temp))
    
df = pd.DataFrame([error])
df.to_csv("WH_EDMDDL.csv", index=False)

In [9]:
error

7.47585868356635e-06